In [1]:
import numpy as np
from lib.envs.bandit import BanditEnv, ActionSpace
from lib.running_variance import RunningVariance
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
from six.moves import xrange
import tensorflow as tf
import tensorflow_constrained_optimization as tfco
from sklearn.metrics import accuracy_score

/opt/conda/envs/ao_proptech/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/ao_proptech/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/ao_proptech/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/ao_proptech/lib/python3.7/site-packages/tensorflow/

In [2]:
df = pd.read_csv("companies_raw_continent.csv")
cols = ["S. No.", "Month", "Stage", "Continent", "HQ", "re1", "re2", "re3", "re4", "bm1", "bm2", "bm3", "bm4", "pv1", 
        "pv2", "pv3", "pv4"]
df.columns = cols
df_outliers = df.where(df['Stage'] == "TBD").dropna(axis=0, how='all')
df.drop(index=df_outliers.index, inplace=True)
y = df.loc[:, ["Stage"]]
y.dropna(axis=0, inplace=True)
X = df.loc[y.index, ["Month", "Continent", "HQ", "re1", "re2", "re3", "re4", "bm1", "bm2", "bm3", "bm4", "pv1", "pv2", "pv3", "pv4"]]

In [3]:
X_dummies = pd.get_dummies(X)
y_dummies = pd.get_dummies(y)
X_train, X_test, y_train, y_test = train_test_split(X_dummies.cumsum(), y, test_size=0.2, random_state=123)

In [5]:
class MinimizationProblem(tfco.ConstrainedMinimizationProblem):
    def __init__(self, labels, predictions, recall_lower_bound):
        self._labels = labels
        self._predictions = predictions
        self._recall_lower_bound = recall_lower_bound
        # The number of positively-labeled examples.
        self._positive_count = tf.reduce_sum(self._labels)

    @property
    def num_constraints(self):
        return 1
    
    # we use hinge loss because we need to capture those that are not classified correctly and minimize that loss
    def objective(self):
        predictions = self._predictions
        if callable(predictions):
            predictions = predictions()
        return tf.compat.v1.losses.hinge_loss(labels=self._labels,
          logits=predictions)
    
    def constraints(self):
        # In eager mode, the predictions must be a nullary function returning a
        # Tensor. In graph mode, they could be either such a function, or a Tensor
        # itself.
        predictions = self._predictions
        if callable(predictions):
            predictions = predictions()
        # Recall that the labels are binary (0 or 1).
        true_positives = self._labels * tf.cast(predictions > 0, dtype=tf.float32)
        accuracy = tf.reduce_sum(self._labels - true_positives)
        # The constraint is (recall >= self._recall_lower_bound), which we convert
        # to (self._recall_lower_bound - recall <= 0) because
        # ConstrainedMinimizationProblems must always provide their constraints in
        # the form (tensor <= 0).
        #
        # The result of this function should be a tensor, with each element being
        # a quantity that is constrained to be non-positive. We only have one
        # constraint, so we return a one-element tensor.
        return accuracy - self._recall_lower_bound
    
    def proxy_constraints(self):
        # In eager mode, the predictions must be a nullary function returning a
        # Tensor. In graph mode, they could be either such a function, or a Tensor
        # itself.
        predictions = self._predictions
        if callable(predictions):
            predictions = predictions()
        # Use 1 - hinge since we're SUBTRACTING recall in the constraint function,
        # and we want the proxy constraint function to be convex. Recall that the
        # labels are binary (0 or 1).
        true_positives = self._labels * tf.minimum(1.0, predictions)
        accuracy = tf.reduce_sum(self._labels - true_positives)
        # Please see the corresponding comment in the constraints property.
        return accuracy - self._recall_lower_bound

In [6]:
np.random.seed(10)

def predictions():
    return tf.tensordot(yro, mean_r, axes=(1, 0)) - threshold

num_examples = 356
num_mislabeled_examples = 0
dimension = 246
class_dimension = 16

accuracy_lower_bound = 356.0

mean_r = tf.Variable(tf.zeros((dimension, class_dimension)), dtype=tf.float32, name="weights")
threshold = tf.Variable(tf.zeros((num_examples, class_dimension)), dtype=tf.float32, name="threshold")

input_data = X_dummies.cumsum().values

labels = y_dummies.values

constant_labels = tf.constant(labels, dtype=tf.float32)
constant_input = tf.constant(input_data, dtype=tf.float32)
yro = constant_input

problem = MinimizationProblem(
    labels=constant_labels,
    predictions=predictions,
    recall_lower_bound=accuracy_lower_bound,
)

In [16]:
if tf.executing_eagerly():
    # In eager mode, we use a V2 optimizer (a tf.keras.optimizers.Optimizer). A V1
    # optimizer, however, would work equally well.
    optimizer = tfco.ProxyLagrangianOptimizerV2(
      optimizer=tf.keras.optimizers.Adagrad(learning_rate=1.0),
      num_constraints=problem.num_constraints)
    # In addition to the model parameters (weights and threshold), we also need to
    # optimize over any trainable variables associated with the problem (e.g.
    # implicit slack variables and weight denominators), and those associated with
    # the optimizer (the analogues of the Lagrange multipliers used by the
    # proxy-Lagrangian formulation).
    var_list = ([mean_r, threshold] + problem.trainable_variables +
              optimizer.trainable_variables())

    for ii in xrange(1000):
        optimizer.minimize(problem, var_list=var_list)

    trained_weights = mean_r.numpy()
    trained_threshold = threshold.numpy()

else:  # We're in graph mode.
    # In graph mode, we use a V1 optimizer (a tf.compat.v1.train.Optimizer). A V2
    # optimizer, however, would work equally well.
    optimizer = tfco.ProxyLagrangianOptimizerV1(
      optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=1.0))
    train_op = optimizer.minimize(problem)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        for ii in xrange(1000):
            session.run(train_op)

        trained_weights, trained_threshold = session.run((mean_r, threshold))

In [17]:
def average_hinge_loss(labels, predictions):
    # Recall that the labels are binary (0 or 1).
    signed_labels = (labels * 2) - 1
    return np.mean(np.maximum(0.0, 1 - signed_labels * (predictions > 0)))

def accuracy(labels, predictions):
    # Recall that the labels are binary (0 or 1).
    true_positives = labels * (predictions > 0)
    return accuracy_score(labels, true_positives)

In [15]:
trained_predictions = np.matmul(input_data, trained_weights) - trained_threshold
print("Constrained average hinge loss = %f" % average_hinge_loss(
    labels, trained_predictions))
print("Constrained accuracy = %f" % accuracy(labels, trained_predictions))
print("Error = %f" % (np.sum(labels - (labels * (trained_predictions > 0))) / len(labels)))

Constrained average hinge loss = 1.022121
Constrained accuracy = 0.873596
Error = 0.126404


### Portion to be Done (Reinforcement Learning)

In [ ]:
state_dim = 246 # Dimension of state space
action_count = 16 # Number of actions
update_frequency = 20

def discount_rewards(r, factor_model, gamma=0.999):
    """Take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r,dtype=np.float32)
    running_add = 0
    f_ = 0
    f = np.linalg.norm(factor_model[0],1)
    running_add = f
    for t in reversed(range(0, r.size)):
        if (t < (r.size - 1) and r.size >= 2):
            f_ = np.linalg.norm(factor_model[t+1],1)
            f = np.linalg.norm(factor_model[t],1)
            running_add = running_add + gamma * f_ - f
        running_add = running_add + r[t]
        discounted_r[t] = running_add
    return discounted_r

def execute(NOISE_PARAM=4.30, num_epochs=1, max_number_of_episodes=500, reward_sum=0):

    class PGCREnv(BanditEnv):
        def __init__(self, num_actions = 10, 
        observation_space = None, distribution = "factor_model", evaluation_seed=387):
            super(BanditEnv, self).__init__()
            
            self.action_space = ActionSpace(range(num_actions))
            self.distribution = distribution
            
            self.observation_space = observation_space
            
            np.random.seed(evaluation_seed)
            
            self.reward_parameters = None
            if distribution == "bernoulli":
                self.reward_parameters = np.random.rand(num_actions)
            elif distribution == "normal":
                self.reward_parameters = (np.random.randn(num_actions), np.random.rand(num_actions))
            elif distribution == "heavy-tail":
                self.reward_parameters = np.random.rand(num_actions)
            elif distribution == "factor_model":
                self.reward_parameters = (np.array(list(factors.values())).sum(axis=2), 
                            np.array(list(noises.values())))
            else:
                print("Please use a supported reward distribution", flush = True)
                sys.exit(0)
            
            if distribution != "normal":
                self.optimal_arm = np.argmax(self.reward_parameters)
            else:
                self.optimal_arm = np.argmax(self.reward_parameters[0])
        
        def reset(self):
            self.is_reset = True
            action = np.random.randint(0,action_count)
            return mel_component_matrix[action], list(factors.keys())[action]
        
        def compute_gap(self, action):
            if self.distribution == "factor_model":
                gap = np.absolute(self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action])
            elif self.distribution != "normal":
                gap = np.absolute(self.reward_parameters[self.optimal_arm] - self.reward_parameters[action])
            else:
                gap = np.absolute(self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action])
            return gap
        
        def step(self, action):
            self.is_reset = False
            
            valid_action = True
            if (action is None or action < 0 or action >= self.action_space.n):
                print("Algorithm chose an invalid action; reset reward to -inf", flush = True)
                reward = float("-inf")
                gap = float("inf")
                valid_action = False
            
            if self.distribution == "bernoulli":
                if valid_action:
                    reward = np.random.binomial(1, self.reward_parameters[action])
                    gap = self.reward_parameters[self.optimal_arm] - self.reward_parameters[action]
            elif self.distribution == "normal":
                if valid_action:
                    reward = self.reward_parameters[0][action] + self.reward_parameters[1][action] * np.random.randn()
                    gap = self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action]
            elif self.distribution == "heavy-tail":
                if valid_action:
                    reward = self.reward_parameters[action] + np.random.standard_cauchy()
                    gap = self.reward_parameters[self.optimal_arm] - self.reward_parameters[action]        #HACK to compute expected gap
            elif self.distribution == "factor_model":
                if valid_action:
                    reward = np.linalg.norm(
                        self.reward_parameters[0][action],1
                    ) + \
                    np.linalg.norm(
                        self.reward_parameters[1][action],1
                    ) * np.random.randn()
            else:
                print("Please use a supported reward distribution", flush = True)
                sys.exit(0)
                
            observation = marginal_model(action)
            
            return(observation, list(factors.keys())[action], reward, self.is_reset, '')

    running_variance = RunningVariance()

    epoch_stats = []
    net_actions = []
    net_rewards = []
    net_scores = []

    for epoch in tqdm(range(num_epochs)):
        stats = plotting.EpisodeStats(
            episode_lengths=np.zeros(max_number_of_episodes),
            episode_rewards=np.zeros(max_number_of_episodes),
            episode_running_variance=np.zeros(max_number_of_episodes),
            episode_scores=np.zeros(max_number_of_episodes),
            losses=np.zeros(max_number_of_episodes))

        env = PGCREnv(num_actions = action_count, observation_space = np.zeros((state_dim,1)))
        
        for episode_number in tqdm(range(max_number_of_episodes)):
            states, rewards, labels, scores = [],[],[],[]
            done = False

            observation, model = env.reset()
            factor_sequence = []
            t = 1
            for state_dim_i in range(state_dim):
                done = False
                while not done:
                    
                    state = np.ascontiguousarray(np.reshape(observation[:,state_dim_i], [1,120]).astype(np.float32))
                    states.append(state)

                    is_reset = False
                    score = 0.0
                    action, score = least_noise_model(
                        model, non_deterministic_hierarchical_clustering(model), 
                        scaler, MAX_CLUSTERS, NOISE_PARAM, noise_mean, noise_std, 
                        env=env
                    )
                    is_reset = env.is_reset
                    
                    net_actions.append(action)

                    z = np.ones((1,state_dim)).astype(np.float32) * 1.25/120
                    z[:,state_dim_i] = 0.75
                    labels.append(z)
                    
                    # step the environment and get new measurements
                    observation, model, reward, done, _ = env.step(action)
                    
                    done = is_reset if is_reset is True else False

                    observation = np.ascontiguousarray(observation)

                    net_rewards.append(reward)
                    net_scores.append(score)

                    reward_sum += float(reward)

                    # Record reward (has to be done after we call step() to get reward for previous action)
                    rewards.append(float(reward))

                    factor_sequence.append(list(factors.values())[action])

                    stats.episode_rewards[episode_number] += reward
                    stats.episode_lengths[episode_number] = t
                    stats.episode_scores[episode_number] += score

                    t += 1

            # Stack together all inputs, hidden states, action gradients, and rewards for this episode
            epr = np.vstack(rewards).astype(np.float32)

            # Compute the discounted reward backwards through time.
            discounted_epr = discount_rewards(epr, factor_sequence)
            
            for discounted_reward in discounted_epr:
                # Keep a running estimate over the variance of of the discounted rewards
                running_variance.add(discounted_reward.sum())

            stats.episode_running_variance[episode_number] = running_variance.get_variance()
        
        plotting.plot_pgresults(stats)
        epoch_stats.append(stats)

    uniq_actions = np.unique(net_actions)
    np.save(dirname + "/tmp_models/net_actions.npy", uniq_actions)

    print("MCTS Coverage: ", len(uniq_actions) / state_dim)

    return stats